# Blending Problem

We’re going to make some sausages!

We have the following ingredients available to us:

| **Ingredient** | **Cost (€/kg)** | **Availability (kg)** |
| --- | --- |--- |
| Pork | 4.32 | 30 |
| Wheat	| 2.46 | 20 |
| Starch | 1.86	| 17 |

We’ll make 2 types of sausage:
* Economy (>40% Pork)
* Premium (>60% Pork)

One sausage is 50 grams (0.05 kg).

According to government regulations, the most starch we can use in our sausages is 25%.

We have a contract with a butcher, and have already purchased 23 kg pork, that must go in our sausages.

We have a demand for 350 economy sausages and 500 premium sausages.

We need to figure out how to most cost effectively blend our sausages.

# Modelling Problem

Let’s model our problem.

**Variables**

* Pork in Economy Sausage in kg = p1
* Wheat in Economy Sausage in kg = w1
* Starch in Economy Sausage in kg = s1
* Pork in Premium Sausage in kg = p2
* Wheat in Premium Sausage in kg = w2
* Starch in Premium Sausage in kg = s2

**Objective function:** We want to minimize sausage manufacturing cost.

Minimize z = 4.32 (p1 + p2) + 2.46 (w1 + w2) + 1.86 (s1 + s2)

**Constraints:**

| Constraints | Reasons |
| :- | :- |
| p1 + w1 + s1 = 350×(0.05) | 350 economy sausage demand |
| p2 + w2 + s2 = 500×(0.05) | 500 premium sausage demand |
| p1 > 0.4×(p1 + w1 + s1) | pork in economy sausage should be greater than 40% |
| p2 > 0.6×(p2 + w2 + s2) | pork in premium sausage should be greater than 60% |
| s1 <= 0.25×(p1 + w1 + s1) | atmost 25% starch can be used |
| s2 <= 0.25×(p2 + w2 + s2) | atmost 25% starch can be used |
| p1 + p2 >= 23 | 23 Kg pork has been already purchased |
| p1 + p2 <= 30 | pork availability |
| w1 + w2 <= 20 | wheat availability |
| s1 + s2 <= 17 | starch availability |

# Solving LPP

In [1]:
import pulp as p

In [2]:
# initiating model instance
model = p.LpProblem("Cost-Minimizing-Blending-Problem", sense = p.LpMinimize)

In [3]:
# type of sausage
s_type = ['economy', 'premium']

# ingredients
s_ingre = ['pork', 'wheat', 'starch'] 

# defining weight variables
weight = p.LpVariable.dicts("Weight_kg",
                            ((i,j) for i in s_type for j in s_ingre),
                            lowBound = 0)

In [4]:
# Objective Function
model += (
            p.lpSum([
                        4.32*weight[i,'pork']
                        + 2.46*weight[i,'wheat']
                        + 1.86*weight[i,'starch']
                        for i in s_type
                    ])
         )

In [5]:
# Adding constraints
model += p.lpSum([weight['economy', j] for j in s_ingre]) == 350*0.05
model += p.lpSum([weight['premium', j] for j in s_ingre]) == 500*0.05

model += weight['economy','pork'] >= 0.4*(p.lpSum([weight['economy', j] for j in s_ingre]))
model += weight['premium','pork'] >= 0.6*(p.lpSum([weight['premium', j] for j in s_ingre]))

model += weight['economy','starch'] <= 0.25*(p.lpSum([weight['economy', j] for j in s_ingre]))
model += weight['premium','starch'] <= 0.25*(p.lpSum([weight['premium', j] for j in s_ingre]))

model += p.lpSum([weight[i,'pork'] for i in s_type]) >=23
model += p.lpSum([weight[i,'pork'] for i in s_type]) <=30
model += p.lpSum([weight[i,'wheat'] for i in s_type]) <=20
model += p.lpSum([weight[i,'starch'] for i in s_type]) <=17

In [6]:
# printing problem
print(model)

Cost-Minimizing-Blending-Problem:
MINIMIZE
4.32*Weight_kg_('economy',_'pork') + 1.86*Weight_kg_('economy',_'starch') + 2.46*Weight_kg_('economy',_'wheat') + 4.32*Weight_kg_('premium',_'pork') + 1.86*Weight_kg_('premium',_'starch') + 2.46*Weight_kg_('premium',_'wheat') + 0.0
SUBJECT TO
_C1: Weight_kg_('economy',_'pork') + Weight_kg_('economy',_'starch')
 + Weight_kg_('economy',_'wheat') = 17.5

_C2: Weight_kg_('premium',_'pork') + Weight_kg_('premium',_'starch')
 + Weight_kg_('premium',_'wheat') = 25

_C3: 0.6 Weight_kg_('economy',_'pork') - 0.4 Weight_kg_('economy',_'starch')
 - 0.4 Weight_kg_('economy',_'wheat') >= 0

_C4: 0.4 Weight_kg_('premium',_'pork') - 0.6 Weight_kg_('premium',_'starch')
 - 0.6 Weight_kg_('premium',_'wheat') >= 0

_C5: - 0.25 Weight_kg_('economy',_'pork')
 + 0.75 Weight_kg_('economy',_'starch') - 0.25 Weight_kg_('economy',_'wheat')
 <= 0

_C6: - 0.25 Weight_kg_('premium',_'pork')
 + 0.75 Weight_kg_('premium',_'starch') - 0.25 Weight_kg_('premium',_'wheat')
 <= 0

In [7]:
# model status
status = p.LpStatus[model.solve()]
print(status)

Optimal


In [8]:
for var in weight:
    print(f'In {var[0]} sausage, weight of {var[1]} used = {weight[var].varValue} Kg')

In economy sausage, weight of pork used = 7.0 Kg
In economy sausage, weight of wheat used = 6.125 Kg
In economy sausage, weight of starch used = 4.375 Kg
In premium sausage, weight of pork used = 16.0 Kg
In premium sausage, weight of wheat used = 2.75 Kg
In premium sausage, weight of starch used = 6.25 Kg


In [12]:
print(f'The minimised cost of production = €{p.value(model.objective)}')

The minimised cost of production = €140.955
